In [ ]:
import os, platform, subprocess, sys, requests, json
import pandas as pd
from pandas import json_normalize
import matplotlib.pyplot as plt

In [ ]:
"""
Finding and importing provided libraries regardless of system and platform type
"""
## find and import libraries relative to this file's location (regardless of script or notebook)
## platfrom specific naming already handled within the lib directory's __init__ file
if '__file__' not in globals():
    sys.path.append(os.path.dirname(os.path.abspath('').split('transactions')[0]))
else:
    sys.path.append(os.path.dirname(__file__).split('transactions')[0])

from lib.encoder  import encode_tx, encode_script
from lib.helper   import decode_address
from lib.hash     import hash160, hash256, sha256
from lib.rpc      import RpcSocket
from lib.rpcauth  import rpcauth
from lib.sign     import sign_tx

In [ ]:
'''
Step 0.0: setup our RPC socket like we did in the previous example
        If you have not done the previous example, now would be the time.
        Please copy in the variables you used (located conveniently at the end of the previous example)
'''

nodeIP = "127.0.0.1"
nodePort = "18444"
username = "user"
password = "password"
wallet = "test2"

rpc = RpcSocket({ 'wallet': wallet,
                  'username': username,
                  'password': password,
                  'url' : nodeIP,
                  'port': nodePort})


In [ ]:
'''
Get the latest block
'''
best_block_hash = rpc.getChain()['bestblockhash']
print(best_block_hash)

In [ ]:
'''
Get a previous block given an existing block hash (using latest from prior step)
'''
rpc.getBlockHeader(best_block_hash)['previousblockhash']

In [ ]:
'''
Get info on the block
'''
rpc.getBlock(best_block_hash)

In [ ]:
'''
Get stats on the block
'''
rpc.getBlockStats(best_block_hash)

In [ ]:
'''
Function to iterate backwards through 'N' number of latest blocks in the chain
'''
def getBlockHashList(count):
    if count < 1:
        return []
    chain_height = rpc.getBlockCount()
    if chain_height < count:
        max_block_count = chain_height
    else:
        max_block_count = count
    cur_block_count = 1
    cur_block_hash = rpc.getChain()['bestblockhash']
    block_hash_list = [cur_block_hash]

    while cur_block_count < max_block_count:
        prev_block_hash = rpc.getBlockHeader(cur_block_hash)['previousblockhash']
        cur_block_count += 1
        block_hash_list.append(prev_block_hash)
        cur_block_hash = prev_block_hash

    return block_hash_list

In [ ]:
'''
Get the hashes the top N blocks
'''
block_list = getBlockHashList(200)

In [ ]:
'''
Converting the block info into a pandas dataframe
'''

price_base_url = 'https://min-api.cryptocompare.com/data/pricehistorical?fsym=BTC&tsyms=USD&ts='

df = pd.DataFrame()
for block_hash in block_list:
    block = rpc.getBlock(block_hash)
    stats = rpc.getBlockStats(block_hash)
    price = requests.get(price_base_url + str(block['time'])).json()
    block.update(stats)
    block.update({'price_usd':price['BTC']['USD']})
    df = pd.concat([pd.DataFrame.from_dict(block, orient='index').T, df], axis=0, join='outer', ignore_index=True)
df

In [ ]:
'''
Convert fees to USD
'''

df['total_fee_usd'] = df.apply(lambda row: (row.totalfee / 100000000) * row.price_usd, axis=1)
df['avg_fee_usd'] = df.apply(lambda row: (row.avgfee / 100000000) * row.price_usd, axis=1)
df['max_fee_usd'] = df.apply(lambda row: (row.maxfee / 100000000) * row.price_usd, axis=1)
df['min_fee_usd'] = df.apply(lambda row: (row.minfee / 100000000) * row.price_usd, axis=1)
df['median_fee_usd'] = df.apply(lambda row: (row.medianfee / 100000000) * row.price_usd, axis=1)


In [ ]:
print('shape of table is:', df.shape)
print(df.columns)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
def getTransactionIDList(df):
    tx_list = df["tx"]
    tx_list_flattened = [item for items in tx_list for item in items]
    return tx_list_flattened

In [ ]:
getTransactionIDList(df)

In [ ]:
df["time"]

In [ ]:
plt.figure();

rel_time = []
for idx, x in enumerate(df["time"]):
    if idx == 0:
        continue
    rel_time.append(df["time"][idx] - df["time"][idx-1])

rel_time = pd.DataFrame(rel_time)

rel_time.plot();
plt.xlabel("Blocks")
plt.ylabel("Time in s")

In [ ]:
df.plot(y=["avgfee", "medianfee"])
plt.xlabel("Blocks")
plt.ylabel("Fee in Sats")

In [ ]:
df.plot(y=["avg_fee_usd", "median_fee_usd"])
plt.xlabel("Blocks")
plt.ylabel("Fee in USD")

In [ ]:
df.plot(y=["nTx"])
plt.xlabel("Blocks")
plt.ylabel("# of Transactions")

In [ ]:
df.plot(y=["total_out"])
plt.xlabel("Blocks")
plt.ylabel("Total Sats moved")

In [ ]:
df.plot(y=["total_size"])
plt.xlabel("Blocks")
plt.ylabel("Block Size")